In [1]:
from transformers import AutoTokenizer, AutoModel
tokenizer = AutoTokenizer.from_pretrained('dnagpt/human_gpt2-v1')
tokenizer.pad_token = tokenizer.eos_token

/home/liming/anaconda3/envs/pytorch/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
full_model = AutoModel.from_pretrained('dnagpt/human_gpt2-v1')

In [4]:
gena_module_name = full_model.__class__.__module__
print(gena_module_name)

transformers.models.gpt2.modeling_gpt2


In [5]:
import importlib
myclass = importlib.import_module(gena_module_name)
#dir(myclass)

In [6]:
cls = getattr(importlib.import_module(gena_module_name), 'GPT2ForSequenceClassification')
cls

transformers.models.gpt2.modeling_gpt2.GPT2ForSequenceClassification

In [7]:
model = cls.from_pretrained('dnagpt/human_gpt2-v1', num_labels=2)
model.config.pad_token_id = model.config.eos_token_id

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at dnagpt/human_gpt2-v1 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
from datasets import load_dataset
# load ~11k samples from promoters prediction dataset
dataset = load_dataset("yurakuratov/example_promoters_2k")['train'].train_test_split(test_size=0.1)

In [9]:
dataset

DatasetDict({
    train: Dataset({
        features: ['sequence', 'promoter_presence'],
        num_rows: 10656
    })
    test: Dataset({
        features: ['sequence', 'promoter_presence'],
        num_rows: 1184
    })
})

In [10]:
dataset['train'][0]

{'sequence': 'TTGATATGCCCGCAATAAATGTTAGCCCTTCCTCTTTACAAACAATTATTATTTCATTAAGACTTACTCCTGGGAAGAAGTCACACTTTTCTGTGTGCTAACATCATGTACATTCTGATTCTCTGTCCTGTGTGCCATTCCATCATGCTCTCTACCAGCAAAACGCCACCAGTCTAAGGAGGAGACAGAACCGCATAAAAATAGTCACACAGTCTTCAGAGCTGCACAGGGAGTGGCACAGGGGCTTCACAGGCCTTGGGAAGTTTGTAAAGTGAGGGAGAAGCACTTTACTGTCGCCAGCTCCATTGAAAACAGATTGTAGCAAATGACTTCCCAGGAGCTCCTCCTCTCACCTTGGTGAATGTTGGGAGAGAATGAAGACTGCGGCTGAGAGGCAGCGCCTTGAGCCTTGAACGGAAAGAGGGACAGGGCCTGGGGGCTGTCACTGAGGTGCAGAAATGGGTTTTGGGCTGGGCATCAGACAACCCCAGATCCCAACACTGGCTCAGCCAATTCCTGGCTGTGTGACATGGAGCCATTACTGTACTTTCCTTGCCAGGCCTCCATTTTTCCTCTCTCTAGAATGGGGAGGCTGGTAATACCCACTTTGGAGGGCAGTGTGAGGACCGAAGGGGGCTGTTAAGTGCTCGGCGCGCAGCGAGCACTCAGGACGGGTGGCAGCGGTCACCTTTGTCGCCACCGTCCAAGCCTTTGCAGAGGCTCAGGGCCGAGCCCCGGACATGCGCATGCGCACGCGCACCCGGCCGGCCAGGTGAGAAGGAGGCGGCCGCCTCGCACCTGGGCTTTAACCCTTCCCAGTCACGGGAGTGGGCGGAGTGCACGCAGGGACGGAGTCGGGGGCGAGCAGCGAGAGAGGCCGGGCTTGACCCTTGAGGCGGGGTGGGCAGCGTCCGGGGAGCGCGCCGAGCCGCTTCTAGCCTGGGGTCCGCGCGCGCCGCGGGGAGGAGGGGGAGTGCCGAGGGGAG

In [11]:
print('# base pairs: ', len(dataset['train'][0]['sequence']))

# base pairs:  2000


In [12]:
print('tokens: ', ' '.join(tokenizer.tokenize(dataset['train'][0]['sequence'])))

tokens:  TTGATATG CCCGC AATAAATG TTAGCCC TTCCTC TTTACAAAC AATTATT ATTTCATT AAGAC TTAC TCCTGGG AAGAAGTC ACAC TTTTC TGTGTGC TAAC ATCATG TAC ATTCTG ATTC TCTGTCC TGTGTGCC ATTCC ATCATGC TCTC TACCAGC AAAAC GCC ACCAGTC TAAGG AGGAGAC AGAACC GC ATAAAA ATAGTC ACAC AGTCTTC AGAGC TGCAC AGGG AGTGGCAC AGGGGC TTCAC AGGCC TTGGGAAG TTTGTAA AGTGAGGG AGAAGC ACTTTAC TGTCGCC AGCTCC ATTGAAAAC AGATTG TAGCAAATG ACTTCCC AGGAGC TCCTCC TCTCACC TTGGTG AATG TTGGGAG AGAATGAAG ACTGC GGC TGAG AGGCAGC GCC TTGAGCC TTGAAC GG AAAG AGGGAC AGGGCC TGGGGGC TGTCAC TGAGG TGCAGAA ATGGG TTTTGGGC TGGGC ATCAGAC AACCCC AGATCCC AACACTGGC TCAGCC AATTCC TGGC TGTGTGAC ATGGAGCC ATTACTG TACTTTCC TTGCC AGGCC TCCATT TTTCC TCTCTC TAGAA TGGGG AGGCTGG TAA TACCC ACTTTGG AGGGC AGTGTG AGGACCG AAGG GGGCTG TTAAG TGCTC GGC GCGC AGCG AGCACTC AGGAC GGGTGGC AGCGG TCACC TTTGTC GCC ACCGTCC AAGCC TTTGC AGAGGC TC AGGGCCG AGCCCC GGAC ATGCGC ATGCGC ACGC GCACCCGGCC GGCC AGGTG AGAAGG AGGC GGCCGCC TCGC ACCTGGGC TTTAACCC TTCCC AGTCAC GGG AGTGGGC GG AGTGCAC GCAG

In [13]:
print('# tokens: ', len(tokenizer.tokenize(dataset['train'][0]['sequence'])))

# tokens:  350


In [14]:
def preprocess_labels(example):
  example['label'] = example['promoter_presence']
  return example

dataset = dataset.map(preprocess_labels)

Map: 100%|██████████| 1184/1184 [00:00<00:00, 10784.01 examples/s]


In [20]:
def preprocess_function(examples):
  # just truncate right, but for some tasks symmetric truncation from left and right is more reasonable
  # set max_length to 128 to make experiments faster
  return tokenizer(examples["sequence"], truncation=True, max_length=256) #max_length 128

In [21]:
tokenized_dataset = dataset.map(preprocess_function, batched=True)

Map: 100%|██████████| 1184/1184 [00:00<00:00, 6994.12 examples/s]


In [22]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [23]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['sequence', 'promoter_presence', 'label', 'input_ids', 'attention_mask'],
        num_rows: 10656
    })
    test: Dataset({
        features: ['sequence', 'promoter_presence', 'label', 'input_ids', 'attention_mask'],
        num_rows: 1184
    })
})

In [24]:
from transformers import TrainingArguments, Trainer
import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {'accuracy': (predictions==labels).sum() / len(labels)}

# change training hyperparameters to archive better quality
training_args = TrainingArguments(
    output_dir="test_run",
    learning_rate=1e-4,
    lr_scheduler_type="constant_with_warmup",
    warmup_ratio=0.1,
    optim='adamw_torch',
    weight_decay=0.0,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=5,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="epoch",
    load_best_model_at_end=True
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [26]:
trainer.train()

/home/liming/anaconda3/envs/pytorch/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss,Accuracy
1,0.573000,0.462363,0.778716
2,0.360200,0.504239,0.760135
3,0.201600,0.529274,0.795608
4,0.103800,0.946800,0.784628
5,0.064900,1.108120,0.744932


/home/liming/anaconda3/envs/pytorch/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/home/liming/anaconda3/envs/pytorch/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/home/liming/anaconda3/envs/pytorch/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/home/liming/anaconda3/envs/pytorch/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: Us

TrainOutput(global_step=835, training_loss=0.2607016295016169, metrics={'train_runtime': 499.203, 'train_samples_per_second': 106.73, 'train_steps_per_second': 1.673, 'total_flos': 6960945435770880.0, 'train_loss': 0.2607016295016169, 'epoch': 5.0})